<a href="https://colab.research.google.com/github/EricRibeiroAlves/DeteccaoFakeNews/blob/main/Modelos/%5BAP_T%5D_TI_Modelo_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação de Libs
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# Importação do Dataset ISOT Fake News Dataset
drive.mount('/content/drive')
dat_fake = "/content/drive/MyDrive/Eng. Controle e Automação/8º Semestre/AP/dataset_FakeNews/Fake.csv"
dat_real = "/content/drive/MyDrive/Eng. Controle e Automação/8º Semestre/AP/dataset_FakeNews/True.csv"

# Carregar os dados
dt_fake = pd.read_csv(dat_fake)
dt_real = pd.read_csv(dat_real)

dt_fake['label'] = 'fake'
dt_real['label'] = 'real'

dt = pd.concat([dt_fake, dt_real], ignore_index=True)

# Filtro p/ Variáveis de Interesse
df = dt[['title', 'text', 'label']]

# Verificação de valores nulos
print("Valores ausentes antes do tratamento:")
print(df.isnull().sum())

# Remover linhas com valores nulos
df = df.dropna()

print("Valores ausentes após o tratamento:")
print(df.isnull().sum())

# Codificar os rótulos
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Divisão dos dados
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.3, random_state=42)

# Configurações BERT
modelo_bert = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(modelo_bert)

# Tokenização
def tokenizar_dados(textos, labels=None, max_length=512):
    encodings = tokenizer(
        list(textos),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='tf'
    )

    if labels is not None:
        return tf.data.Dataset.from_tensor_slices((dict(encodings), labels)).batch(16)
    return tf.data.Dataset.from_tensor_slices(dict(encodings)).batch(16)

# Preparar datasets
train_data = tokenizar_dados(x_train, y_train)
test_data = tokenizar_dados(x_test, y_test)

# Criação do modelo BERT
modelo = TFBertForSequenceClassification.from_pretrained(
    modelo_bert,
    num_labels=2
)

# Compilação do modelo
modelo.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Treinamento do modelo
modelo.fit(train_data, epochs=3)

# Previsão e Resultados
logits = modelo.predict(test_data).logits
y_pred = tf.argmax(logits, axis=1).numpy()

# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, tf.nn.softmax(logits, axis=1)[:, 1].numpy())

print("Métricas do modelo BERT:")
print(f"Acurácia: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# Relatório de classificação detalhado
target_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(y_test, y_pred, target_names=target_names))

# Matriz de Confusão
y_pred_classes = modelo.predict(test_data)
y_pred_classes = tf.argmax(y_pred_classes.logits, axis=1).numpy()
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_classes, display_labels=label_encoder.classes_, cmap='Purples')
plt.show()

# Função para classificar uma notícia
def classificar_noticia(texto_noticia):
    # Validar entrada
    if not texto_noticia.strip():
        return "Texto inválido. Por favor, insira um texto para classificar."

    # Tokenizar a notícia
    texto_tokenizado = tokenizar_dados([texto_noticia])

    # Fazer a previsão com o modelo treinado
    previsao = modelo.predict(texto_tokenizado)
    previsao_classe = tf.argmax(previsao.logits, axis=1).numpy()

    # Decodificar o rótulo
    resultado = label_encoder.inverse_transform(previsao_classe)

    # Retornar o resultado
    if resultado[0] == 'fake':
        return "A notícia é FAKE!"
    else:
        return "A notícia é REAL!"

# Exemplo de uso:
texto_entrada = input("Digite o texto da notícia para classificar: ")

# Classificar a notícia
resultado = classificar_noticia(texto_entrada)

# Mostrar o resultado
print(resultado)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Valores ausentes antes do tratamento:
title    0
text     0
label    0
dtype: int64
Valores ausentes após o tratamento:
title    0
text     0
label    0
dtype: int64


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
